# ALeRCE DB examples

In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#!pip install psycopg2-binary
import psycopg2

# Get the credentials and open a database connection

In [3]:
import requests
url = "https://raw.githubusercontent.com/alercebroker/usecases/master/alercereaduser_v4.json"
params = requests.get(url).json()['params']

In [4]:
conn = psycopg2.connect(dbname=params['dbname'], user=params['user'], host=params['host'], password=params['password'])

# Query all the table names

In [5]:
query = """
SELECT table_name  FROM information_schema.tables
WHERE table_schema='alerce'
ORDER BY table_name;
"""
tables = pd.read_sql_query(query, conn)
tables.sort_values(by="table_name")

/home/fforster/anaconda3/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,table_name
0,allwise
1,dataquality
2,detection
3,feature
4,feature_version
5,gaia_ztf
6,lc_classifier
7,lc_classifier_periodic
8,lc_classifier_stochastic
9,lc_classifier_top


# Query all the column names for all the available tables

In [6]:
alltabs = []
for tab in sorted(tables.table_name):
    cols = pd.DataFrame()
    query = "select column_name, data_type from information_schema.columns where table_name = '%s';" % tab
    results = pd.read_sql_query(query, conn)
    results["table"] = tab
    alltabs.append(results)
dftab = pd.concat(alltabs)
pd.options.display.max_rows = 999
display(dftab[["table", "column_name", "data_type"]])
pd.options.display.max_rows = 101

/home/fforster/anaconda3/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/fforster/anaconda3/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/fforster/anaconda3/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/fforster/anaconda3/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/c

,table,column_name,data_type
0,allwise,k_msig_2mass,double precision
1,allwise,ra,double precision
2,allwise,dec,double precision
3,allwise,w1mpro,double precision
4,allwise,w2mpro,double precision
5,allwise,w3mpro,double precision
6,allwise,w4mpro,double precision
7,allwise,w1sigmpro,double precision
8,allwise,w2sigmpro,double precision
9,allwise,w3sigmpro,double precision


# Example use cases

## SN Ia light curves

Please write a script in PostgreSQL that returns the object identifier, the candidate identifier, the magnitudes, magnitude errors, and band identifiers as a function of time of the objects classified as SNIa with a probability larger than 0.6.

In [7]:
query = '''
SELECT
    oid, candid, mjd, fid, magpsf, sigmapsf
FROM
    detection
WHERE
    oid in 
(
SELECT 
    oid
FROM
    probability
WHERE
    classifier_name='lc_classifier'
    AND class_name IN ('SNIa')
    AND ranking=1
    AND probability > 0.6
)
'''
results = pd.read_sql_query(query, conn)
results

/home/fforster/anaconda3/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,oid,candid,mjd,fid,magpsf,sigmapsf
0,ZTF19abzzhgx,993463714515015005,58747.463715,2,19.475004,0.187442
1,ZTF19abzzhgx,993476333215015012,58747.476331,2,19.355606,0.133503
2,ZTF19abzzhgx,993505164515015005,58747.505162,1,19.435148,0.152134
3,ZTF19abzzhgx,993505623215015003,58747.505625,1,19.432198,0.197824
4,ZTF19abzzhgx,996445013215015005,58750.445012,1,18.764479,0.110302
...,...,...,...,...,...,...
15142,ZTF19aaxzglq,912211410915015010,58666.211412,1,19.850494,0.141498
15143,ZTF19aaxzglq,912256180915015017,58666.256181,2,19.577070,0.109534
15144,ZTF19aaxzglq,915182000915015011,58669.182002,2,19.618975,0.155844
15145,ZTF19aaxzglq,918212070915015008,58672.212072,2,19.674967,0.145963


## Cepheid that are unlikely RRL

Get the objects whose Cepheid probability is larger than 0.4 and its RRL probability is less than 0.2

In [20]:
query = '''
SELECT
    oid
FROM
    probability
WHERE
    classifier_name='lc_classifier'
    AND class_name='RRL'
    AND probability < 0.2
    AND oid IN
(
SELECT
    oid
FROM
    probability
WHERE
    classifier_name='lc_classifier'
    AND class_name='CEP'
    AND probability > 0.4
)
'''
results = pd.read_sql_query(query, conn)
results

/home/fforster/anaconda3/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,oid
0,ZTF17aacyrfh
1,ZTF18abaqvkw
2,ZTF18abjtjlj
3,ZTF18acetiij
4,ZTF18abtvyrt
...,...
4260,ZTF18acmfvvc
4261,ZTF17aabwkaz
4262,ZTF18abvolwu
4263,ZTF18abxxovx
